### ICTP-IAEA WORKSHOOP ON ENVIRONMENTAL MAPPING: Mobilising	Trust	in Measurements	and	Engaging Scientific Citizenry, Trieste, March 2017
---


# III. Fetching data from Safecast API with Python

In [1]:
import pandas as pd
import requests

In [3]:
# Building the url as a long string in one go
url = "https://api.safecast.org/measurements.json?"\
      "distance=100000&"\
      "longitude=141.0337&"\
      "captured_before=2012-04-10&"\
      "latitude=37.4218&"\
      "per_page=2000&"\
      "captured_after=2011-03-10"

In [4]:
url

'https://api.safecast.org/measurements.json?distance=100000&longitude=141.0337&captured_before=2012-04-10&latitude=37.4218&per_page=2000&captured_after=2011-03-10'

In [2]:
# More advanced but cleaner way to generate the url
params = {'distance': 100000, 'latitude': 37.4218, 'longitude': 141.0337,
          'captured_after': '2011-03-10', 'captured_before': '2012-04-10',
          'per_page': 20000}
basis_url = 'https://api.safecast.org/measurements.json?'
url = basis_url + '&'.join(['%s=%s' % (key, value) for (key, value) in params.items()])

In [6]:
# Import requests package
import requests

# Package the request, send the request and catch the response: r
r = requests.get(url)

json_data = r.json()

In [7]:
df = pd.DataFrame(json_data)

In [8]:
df.head()

captured_at channel_id device_id devicetype_id height  \
0  2012-01-18T02:32:50.000Z       None      None          None   None   
1  2012-01-18T02:32:45.000Z       None      None          None   None   
2  2012-01-18T02:32:55.000Z       None      None          None   None   
3  2012-01-18T02:33:05.000Z       None      None          None   None   
4  2012-01-18T02:33:10.000Z       None      None          None   None   

         id   latitude location_name   longitude original_id sensor_id  \
0  15500999  37.414865          None  141.028495        None      None   
1  15500998  37.414992          None  141.028215        None      None   
2  15501000  37.414817          None  141.028568        None      None   
3  15501002  37.414787          None  141.028612        None      None   
4  15501003  37.414777          None  141.028622        None      None   

  station_id unit  user_id    value  
0       None  cpm        1  20283.0  
1       None  cpm        1  19713.0  
2       None  cpm        1  20667.0  
3       None  cpm        1  21206.0  
4       None  cpm        1  21220.0

In [9]:
df.shape

(2000, 15)

In [10]:
# Using pandas to get measurements by user
df.groupby(by='user_id').size()

user_id
1    1888
3     112
dtype: int64

In [23]:
df.to_csv('data/safecast_from_api.csv')

## A word on Big data

In [11]:
f = 'data/safecast_big_data.csv'    # Path to file containing the table

# A quick look at first 5 rows
data_iter = pd.read_csv(f, chunksize=5)
next(data_iter)

Unnamed: 0               captured_at  channel_id  device_id  devicetype_id  \
0           0  2012-01-18T02:32:50.000Z         NaN        NaN            NaN   
1           1  2012-01-18T02:32:45.000Z         NaN        NaN            NaN   
2           2  2012-01-18T02:32:55.000Z         NaN        NaN            NaN   
3           3  2012-01-18T02:33:05.000Z         NaN        NaN            NaN   
4           4  2012-01-18T02:33:10.000Z         NaN        NaN            NaN   

   height        id   latitude  location_name   longitude  original_id  \
0     NaN  15500999  37.414865            NaN  141.028495          NaN   
1     NaN  15500998  37.414992            NaN  141.028215          NaN   
2     NaN  15501000  37.414817            NaN  141.028568          NaN   
3     NaN  15501002  37.414787            NaN  141.028612          NaN   
4     NaN  15501003  37.414777            NaN  141.028622          NaN   

   sensor_id  station_id unit  user_id    value  
0        NaN         NaN  cpm        1  20283.0  
1        NaN         NaN  cpm        1  19713.0  
2        NaN         NaN  cpm        1  20667.0  
3        NaN         NaN  cpm        1  21206.0  
4        NaN         NaN  cpm        1  21220.0

In [12]:
next(data_iter)

Unnamed: 0               captured_at  channel_id  device_id  devicetype_id  \
5           5  2012-01-18T02:33:00.000Z         NaN        NaN            NaN   
6           6  2012-01-18T02:33:15.000Z         NaN        NaN            NaN   
7           7  2012-01-18T02:38:10.000Z         NaN        NaN            NaN   
8           8  2012-01-18T02:38:15.000Z         NaN        NaN            NaN   
9           9  2012-01-18T02:38:05.000Z         NaN        NaN            NaN   

   height        id   latitude  location_name   longitude  original_id  \
5     NaN  15501001  37.414792            NaN  141.028588          NaN   
6     NaN  15501004  37.414760            NaN  141.028643          NaN   
7     NaN  15501063  37.414698            NaN  141.028720          NaN   
8     NaN  15501064  37.414697            NaN  141.028720          NaN   
9     NaN  15501062  37.414697            NaN  141.028720          NaN   

   sensor_id  station_id unit  user_id    value  
5        NaN         NaN  cpm        1  20916.0  
6        NaN         NaN  cpm        1  20809.0  
7        NaN         NaN  cpm        1  20432.0  
8        NaN         NaN  cpm        1  20394.0  
9        NaN         NaN  cpm        1  20504.0

In [13]:
# Some counts:
#   - Total nb. of measurements
#   - Nb. of measurements by user_id

# Re-initialize iterator
data_iter = pd.read_csv(f, chunksize=1000)

nb_lines = 0
counts_by_user = {}

for chunk in data_iter:
    nb_lines += len(chunk)  
    for entry in chunk['user_id']:
        if entry in counts_by_user.keys():
            counts_by_user[entry] += 1
        else:
            counts_by_user[entry] = 1

In [14]:
counts_by_user

{1: 17813, 3: 538, 148: 1649}

### Exercise

Load from Safecast API:

* first `1000` measurements
* taken between `2011-03-10` and `2017-03-08``
* `10km` around `Trieste`

In [24]:
# Tip: To get latitude and longitude coordinates for Trieste use Google geocoder:
import geocoder
g = geocoder.google('Trieste, Italy')
g.latlng

[45.6495264, 13.7768182]